# Implementation of libFM in Keras

This notebook shows how to implement libfm in Keras, and how to use it in the [Talking Data competition on Kaggle](https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection).  The [companion post](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Implementing_Libfm_in_Keras?lang=en) provides context and explanation of how this works.  

I recommend people also read the seminal paper on Factorization Machines:

Steffen Rendle (2010): <i><a href="https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf">Factorization Machines</a></i>, in Proceedings of the 10th IEEE International Conference on Data Mining (ICDM 2010), Sydney, Australia <a href="https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf">PDF</a>

First, let's do some imports

In [1]:
import pandas as pd
import numpy as np
import gc

import mlcrate as mlc
import pickle as pkl

from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2


/home/jfpuget/anaconda3/envs/tf15/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Then let's load some data.  I had concatenated the original train and test_supplement data, and saved it as a feather file, using the mlcrate library.  Any other way will do, as long as data can be loaded in memory!

In [2]:
data = mlc.load('../data/data_small.feather')

In [3]:
data.columns

Index(['app', 'channel', 'click_id', 'device', 'ip', 'is_attributed', 'os',
       'second'],
      dtype='object')

I often use the following helper function to initialise TensorFlow sessions in order to get (almost) reproducible results. I write almost because CuDNN library is not deterministic, hence results are not totally reproducible.  But fixing all random seeds removes most of the variability.

In [4]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os

def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

We will focus on the interactions between os, device and app categories.  We compute an upper bound on the values in each category to prepare for the deep leanring model.

In [5]:
features = ['os', 'device', 'app']
f_size  = [int(data[f].max()) + 1 for f in features]
f_size

[957, 4228, 769]

First step is to count the number of interaction for each feature combination.

In [6]:
X = data.groupby(features)['click_id'].count()
X.head()

os  device  app
0   0       0         604
            19     385493
            46      16545
            49        792
            50       1089
Name: click_id, dtype: int64

We need to compliment this data set with the combinations for which there are no interactions.  One way to do it is to unstack, then stack the result back. 

Let's do it step by step.  unstack() moves the last index (app) to column names.  Missing entries are then filled with 0.

In [7]:
X = X.unstack().fillna(0)
X.head()

app          0    1    2    3    4    5    6    7    8    9   ...   759  760  \
os device                                                     ...              
0  0       604.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
   4         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
   6         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
   8         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   
   12        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0   

app        761  762  763  764  765  766  767  768  
os device                                          
0  0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
   4       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
   6       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
   8       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
   12      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 769 columns]

stack() moves column names back to index. This gives us the dataset we wanted.  We cast the result so that it takes less memory than if using 64 bits floats.

In [8]:
X = X.stack().astype('float32')
X.head()

os  device  app
0   0       0      604.0
            1        0.0
            2        0.0
            3        0.0
            4        0.0
dtype: float32

We see that missing values now appear as 0.  Last data preparation step is to take the log of the counts to make their distribution less skewed towards large values.  We also move the index to columns via reset_index()

In [9]:
X = np.log1p(X).reset_index()
X.head()

,os,device,app,0
0,0,0,0,6.405229
1,0,0,1,0.000000
2,0,0,2,0.000000
3,0,0,3,0.000000
4,0,0,4,0.000000


We rename the count column.

In [10]:
X.columns=features + ['num']
X.head()

,os,device,app,num
0,0,0,0,6.405229
1,0,0,1,0.000000
2,0,0,2,0.000000
3,0,0,3,0.000000
4,0,0,4,0.000000


We can now get the training data for our deep learning model.  The target is the num column.  Note that we do not use the original problem target here, we use interaction counts.  This is a form of unsupervised learning akin to other matrix factorizaiton techniques like PCA or tSVD.

In [11]:
X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')

This dataset is highly imbalanced with less than 1% of non zero entries:

In [12]:
(X.num > 0).mean()

0.00698991895404317

In order to cope with that imbalance we will use sample weights. We assign a much larger weight to non negative rows (31 vs 1).  The choice of the positive sample weight is a bit arbitrary, and could be tuned.  

Using large weights can lead to exploding gradient.  One way to deal with it is to clip the optimizer norm when compiling the model.

In [13]:
w_train = (30 * (y_train > 0).astype('float32') + 1).ravel()

We can now create our model.  The model is a direct implementaiotn of libfm, with a first embedding layer, then a smart computation of all pairwise dot products.  The smartness is not mine, it comes direclty from Rendle paper.  I generalized a bit his model to allow for non categorical input, in which case I use a dense layer to be consistent with categories embeddings.  More explanation can be found on my blog.

The other modification I made is to create a secondary model that outputs the embeddings.  We will use it to retrieve the embeddings once the main model will be trained.

In [14]:
k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

Let's check the main model architecture.

In [15]:
model, model_features = build_model_1(X_train, f_size)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 2)         1914        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

And the secondary model architecture.

In [16]:
model_features.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 2)         1914        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

We can train the main model now.  I tuned the batch size and found that a fairly large size of 2¹⁷ was best.  I know, this would be ridiculous with other deep leanring models, but here it gives the best loss.  This was traing on a 1080 Ti NVIDIA GPU.  We stop training as soon as it does not improve.

In [17]:
n_epochs = 100
P = 17
try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 2**P
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=0, verbose=1)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          sample_weight=w_train,
          callbacks=[earlystopper],
         )

131072
Train on 5045409 samples, validate on 5045409 samples
Epoch 1/100
5045409/5045409 [==============================] - 4s 1us/step - loss: 2.3507 - val_loss: 0.3435
Epoch 2/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 1.4834 - val_loss: 0.2961
Epoch 3/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 1.1209 - val_loss: 0.2832
Epoch 4/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 0.8814 - val_loss: 0.2525
Epoch 5/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 0.6741 - val_loss: 0.2372
Epoch 6/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 0.5501 - val_loss: 0.2279
Epoch 7/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 0.4848 - val_loss: 0.2139
Epoch 8/100
5045409/5045409 [==============================] - 3s 1us/step - loss: 0.4441 - val_loss: 0.1976
Epoch 9/100
5045409/5045409 [==============================] - 3s 1

Once the main model trained we can get the embeddings by preidcitng the secondary model.  This is one beautiful trick if you think of it: we shared layers between models, and we don't directly use the model we trained for predictions!

In [18]:
X_pred = model_features.predict(X_train, batch_size=batch_size)

We should get 1 factors + 1 bias per feature, i.e. 6 in total.

In [19]:
len(X_pred)

6

We can now retrieve the factors and the biases

In [20]:
factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]

X.head()

,os,device,app,num,os_fm_factor_0,os_fm_factor_1,device_fm_factor_0,device_fm_factor_1,app_fm_factor_0,app_fm_factor_1,os_fm_bias,device_fm_bias,app_fm_bias
0,0,0,0,6.405229,0.170492,-0.083319,0.047872,-0.093722,-0.146895,0.175355,0.012788,-0.300488,0.022133
1,0,0,1,0.000000,0.170492,-0.083319,0.047872,-0.093722,0.457369,0.176898,0.012788,-0.300488,0.170095
2,0,0,2,0.000000,0.170492,-0.083319,0.047872,-0.093722,0.511771,0.186729,0.012788,-0.300488,0.263752
3,0,0,3,0.000000,0.170492,-0.083319,0.047872,-0.093722,0.511416,0.196360,0.012788,-0.300488,0.275957
4,0,0,4,0.000000,0.170492,-0.083319,0.047872,-0.093722,0.275622,0.166596,0.012788,-0.300488,0.056378


We can now save this result.  

In [21]:
mlc.save(X, '../data/os_device_app_fm_2.feather')

I then joined it with the original data, using the feature columns as keys.  Adding this and other matrix factorizaiton greatly improved my model.

In [22]:
for pred in X_pred:
    print(pred.shape)

(5045409, 2)
(5045409, 2)
(5045409, 2)
(5045409, 1)
(5045409, 1)
(5045409, 1)
